In [1]:
import sqlite3
from sqlite3 import OperationalError

import numpy as np
import pandas as pd

from os import path

In [95]:
# connect to local database file.
connection = sqlite3.connect('db_dump.db')

# create cursor to execute sql querys with the connected database
cursor = connection.cursor()

In [96]:
connection

In [97]:
# getting list of tables in db.

def run_query(query):
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return(result)

q = "SELECT name FROM sqlite_master WHERE type='table';"
tables = run_query(q)
tables = [x[0] for x in tables]
tables

['customers', 'date', 'markets', 'products', 'transactions']

In [98]:
# testing out functionality of gathering table names.

q_test = 'SELECT * FROM date;'

# have to have an executed cursor to get description.
cursor.execute(q_test)

# create list with lambda of cursor description and mapping index 0 to x and adding to list.
names = list(map(lambda x: x[0], cursor.description))
names

['date', 'cy_date', 'year', 'month_name', 'date_yy_mmm']

In [99]:
# testing out functonality of pandas df from sql table.
df_date = pd.DataFrame(run_query(q_test), columns=names)
df_date

,date,cy_date,year,month_name,date_yy_mmm
0,2017-06-01,2017-06-01,2017,June,17-Jun\r
1,2017-06-02,2017-06-01,2017,June,17-Jun\r
2,2017-06-03,2017-06-01,2017,June,17-Jun\r
3,2017-06-04,2017-06-01,2017,June,17-Jun\r
4,2017-06-05,2017-06-01,2017,June,17-Jun\r
...,...,...,...,...,...
1121,2020-06-26,2020-06-01,2020,June,20-Jun\r
1122,2020-06-27,2020-06-01,2020,June,20-Jun\r
1123,2020-06-28,2020-06-01,2020,June,20-Jun\r
1124,2020-06-29,2020-06-01,2020,June,20-Jun\r


In [7]:
# create fucntion that turns sql table to pandas df.

def make_df(table):
    
    '''
    Use table name from list of tables as user input.
    input must be string.
    '''
    
    q = f'SELECT * FROM {table}'
    cursor = connection.cursor()
    cursor.execute(q)
    result = cursor.fetchall()
    names = list(map(lambda x: x[0], cursor.description))
    data = pd.DataFrame(result, columns=names )
    cursor.close()
    return data

In [8]:
# test pd df in dictionary
test_dict = {}
test_dict['date'] = make_df('date')
test_dict['date']

,date,cy_date,year,month_name,date_yy_mmm
0,2017-06-01,2017-06-01,2017,June,17-Jun\r
1,2017-06-02,2017-06-01,2017,June,17-Jun\r
2,2017-06-03,2017-06-01,2017,June,17-Jun\r
3,2017-06-04,2017-06-01,2017,June,17-Jun\r
4,2017-06-05,2017-06-01,2017,June,17-Jun\r
...,...,...,...,...,...
1121,2020-06-26,2020-06-01,2020,June,20-Jun\r
1122,2020-06-27,2020-06-01,2020,June,20-Jun\r
1123,2020-06-28,2020-06-01,2020,June,20-Jun\r
1124,2020-06-29,2020-06-01,2020,June,20-Jun\r


In [12]:
# simple query running function.

# dont really need unless running querys:
def run_query(query):
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return(result)

def make_df_dict(db):
    
    '''
    Function: make_df_dict converts sqlite database to dictonary containing all tables.
    
    Input:
        db: is string representation of path to datebase.
        Example -> 'db_dump.db' (file extention = *.db)
    
    Functonality:
        function takes in connection object, gets all table names, column names of each 
        table. creates pandas DataFrame from table with corresponding column names.
        adds DataFrame object to dictonary with key equal to name of the table. 

    Return:
        Dictonary returned containing tables from sql database.
    '''
    
    if path.exists(db):
    
        connection = sqlite3.connect(db)

        data_dict = {}

        cursor = connection.cursor()

        # get table names to itter
        q_tables = "SELECT name FROM sqlite_master WHERE type='table';"
        cursor.execute(q_tables)
        tables = cursor.fetchall()
        tables = [x[0] for x in tables]

        # itter tables/make pandas object from each table.
        for i in tables:
            q = f'SELECT * FROM {i}'
            cursor = connection.cursor()
            cursor.execute(q)
            result = cursor.fetchall()
            names = list(map(lambda x: x[0], cursor.description))
            data_dict[i] = pd.DataFrame(result, columns=names )
            
    else:
        print('Check Path!')

        
    return data_dict

In [14]:
data_dict = make_df_dict('db_dump.db')
data_dict.keys()

dict_keys(['customers', 'date', 'markets', 'products', 'transactions'])

In [100]:
path.exists('db_dump.db')

True

In [45]:
data_dict['transactions']

,product_code,customer_code,markets_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
1,Prod001,Cus002,Mark002,2018-05-08,3,-1.0,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875.0,INR
3,Prod002,Cus003,Mark003,2018-04-11,1,583.0,INR
4,Prod002,Cus004,Mark003,2018-06-18,6,7176.0,INR
...,...,...,...,...,...,...,...
150278,Prod339,Cus005,Mark004,2019-04-18,1,394.0,INR\r
150279,Prod339,Cus020,Mark004,2019-04-23,1,667.0,INR\r
150280,Prod339,Cus007,Mark004,2019-04-23,1,625.0,INR\r
150281,Prod339,Cus006,Mark004,2019-04-24,7,8625.0,INR\r


In [115]:
data_dict['transactions'].rename(columns={'market_code':'markets_code'}, inplace=True)
df_transactions = data_dict['transactions']
df_products = data_dict['products']
df = pd.merge(df_transactions, df_products, on='product_code', how='outer')
df = pd.merge(df, data_dict['customers'], on='customer_code', how='outer')
df = pd.merge(df, data_dict['markets'], on='markets_code', how='outer')
df.rename(columns={'order_date':'date'}, inplace=True)
df = pd.merge(df, data_dict['date'], on='date', how='outer')

In [116]:
df

,product_code,customer_code,markets_code,date,sales_qty,sales_amount,currency,product_type,custmer_name,customer_type,markets_name,zone,cy_date,year,month_name,date_yy_mmm
0,Prod001,Cus001,Mark001,2017-10-10,100.0,41241.0,INR,Own Brand\r,Surge Stores,Brick & Mortar,Chennai,South,2017-10-01,2017.0,October,17-Oct\r
1,Prod001,Cus001,Mark001,2017-10-10,100.0,41241.0,INR\r,Own Brand\r,Surge Stores,Brick & Mortar,Chennai,South,2017-10-01,2017.0,October,17-Oct\r
2,Prod013,Cus001,Mark001,2017-10-10,240.0,143560.0,INR,Own Brand\r,Surge Stores,Brick & Mortar,Chennai,South,2017-10-01,2017.0,October,17-Oct\r
3,Prod013,Cus001,Mark001,2017-10-10,240.0,143560.0,INR\r,Own Brand\r,Surge Stores,Brick & Mortar,Chennai,South,2017-10-01,2017.0,October,17-Oct\r
4,Prod020,Cus001,Mark001,2017-10-10,140.0,153019.0,INR\r,Own Brand\r,Surge Stores,Brick & Mortar,Chennai,South,2017-10-01,2017.0,October,17-Oct\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150600,NaN,NaN,NaN,2020-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,2020.0,June,20-Jun\r
150601,NaN,NaN,NaN,2020-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,2020.0,June,20-Jun\r
150602,NaN,NaN,NaN,2020-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,2020.0,June,20-Jun\r
150603,NaN,NaN,NaN,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,2020.0,June,20-Jun\r


In [106]:
# look for fat
df.isnull().sum()

product_code       322
customer_code      322
markets_code       320
date                 2
sales_qty          322
sales_amount       322
currency           322
product_type     55481
custmer_name       322
customer_type      322
markets_name       320
zone               320
cy_date              2
year                 2
month_name           2
date_yy_mmm          2
dtype: int64

In [119]:
# trim fat
df = df[:-322]
df.isnull().sum()

product_code         0
customer_code        0
markets_code         0
date                 0
sales_qty            0
sales_amount         0
currency             0
product_type     55052
custmer_name         0
customer_type        0
markets_name         0
zone                 0
cy_date              0
year                 0
month_name           0
date_yy_mmm          0
dtype: int64

In [ ]:
# get price of product